In [ ]:
import os

from easyCore.Fitting.Fitting import Fitter
from easyDiffractionLib import Phases
from easyDiffractionLib.Jobs import Powder1DCW
from easyDiffractionLib.interface import InterfaceFactory as Calculator
from easyDiffractionLib.Profiles.P1D import PDFParameters
from easyDiffractionLib.Interfaces.pdffit2 import readGRData

In [ ]:
# plotting
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
output_notebook()
FIGURE_WIDTH = 900
FIGURE_HEIGHT = 300

In [ ]:
calculator = Calculator()
calculator.switch("Pdffit2")


In [ ]:
data_fname = os.path.realpath('Ni-xray.gr')
data = readGRData(data_fname)
cif_fname = os.path.realpath('Ni.cif')
phases = Phases.from_cif_file(cif_fname)

In [ ]:
parameters = PDFParameters()

In [ ]:
job = Powder1DCW('Ni', parameters=parameters, phases=phases, interface=calculator)

fitter = Fitter(job, calculator.fit_func)

In [ ]:
parameters = job.parameters
parameters.qmax = 30
parameters.qdamp = 0.063043
parameters.wavelength = 1.9122
parameters.delta2 = 2.253193
parameters.delta1 = 0.0
parameters.qbroad = 0.1
# let's limit the range of qbroad
parameters.qbroad.min = 0.0
parameters.qbroad.max = 0.5

pattern = job.pattern
job.phases[0].atoms[0].adp.Uiso = 0.005445
job.phases[0].scale = 0.366013
job.phases[0].cell.length_a = 3.52

In [ ]:
x_data = data[:, 0]

# profile calculation
y_data = job.create_simulation(x_data)

In [ ]:
# params to optimize
job.phases[0].cell.length_a.fixed = False
job.phases[0].scale.fixed = False
parameters.qdamp.fixed = False
parameters.qbroad.fixed = False
parameters.delta1.fixed = False
parameters.delta2.fixed = False

In [ ]:
result = fitter.fit(x_data, data[:, 1], 
                    method='least_squares', minimizer_kwargs={'diff_step': 1e-5})

In [ ]:
print("The fit has been successful: {}".format(result.success))  
print("The gooodness of fit (chi2) is: {}".format(result.reduced_chi))
print("The optimized parameters are:")
for param in job.get_fit_parameters():
    print("{}: {}".format(param.name, param.raw_value))

In [ ]:
y_data = calculator.fit_func(x_data)

In [ ]:

# obtain data from PdfFit calculator object
Gobs = data[:, 1]
Gfit = y_data
Gdiff = Gobs - Gfit
Gdiff_baseline = -10

Gdiff_show = Gdiff/5.0 + Gdiff_baseline

In [ ]:
fig = figure()
fig.xaxis.axis_label = 'r (Å)'
fig.yaxis.axis_label = r"$$G (Å^{-2})\$$"
fig.title.text = 'Fit of nickel to x-ray experimental PDF'

fig.line(x_data, Gobs, legend_label='G(r) Data', color='steelblue', line_width=2)
fig.line(x_data, Gfit, legend_label='G(r) Fit', color='orangered', line_width=2)
fig.line(x_data, Gdiff_show, legend_label='G(r) Diff', color='grey', line_width=2)
show(fig)
